Some basic info:

- 19 Actions : All actions start with Action. Examples: Action.Left, Action.Shot
- 0,0 is the field center
- Attack to right, always
- We are the left team, always

- Bottom left/right corner of the field is located at [-1, 0.42] and [1, 0.42], respectively.
- Top left/right corner of the field is located at [-1, -0.42] and [1, -0.42], respectively.
- Left/right goal is located at -1 and 1 X coordinate, respectively. They span between -0.044 and 0.044 in Y coordinates.

- X is for left (my goal) -1 ====>  0   =====> +1 right(other goal)
- Y is for up(0.42)/down(-0.42)


![Actions](https://miro.medium.com/max/1050/1*iRjKtZsw1bBWDpPqbMP74w.png)

More info:

https://github.com/google-research/football/blob/master/gfootball/doc/observation.md

    

# Don't Change

In [ ]:
# Install:
# Kaggle environments.
!git clone https://github.com/Kaggle/kaggle-environments.git
!cd kaggle-environments && pip install .

# GFootball environment.
!apt-get update -y
!apt-get install -y libsdl2-gfx-dev libsdl2-ttf-dev

# Make sure that the Branch in git clone and in wget call matches !!
!git clone -b v2.6 https://github.com/google-research/football.git
!mkdir -p football/third_party/gfootball_engine/lib

!wget https://storage.googleapis.com/gfootball/prebuilt_gameplayfootball_v2.6.so -O football/third_party/gfootball_engine/lib/prebuilt_gameplayfootball.so
!cd football && GFOOTBALL_USE_PREBUILT_SO=1 pip3 install .

# Edit here:

In [ ]:
%%writefile submission.py
from kaggle_environments.envs.football.helpers import *
from math import sqrt

directions = [
[Action.TopLeft, Action.Top, Action.TopRight],
[Action.Left, Action.Idle, Action.Right],
[Action.BottomLeft, Action.Bottom, Action.BottomRight]]

dirsign = lambda x: 1 if abs(x) < 0.01 else (0 if x < 0 else 2)

enemyGoal = [1, 0]  


def inside(pos, area):
    return area[0][0] <= pos[0] <= area[0][1] and area[1][0] <= pos[1] <= area[1][1]


def get_distance(pos1,pos2):
    return ((pos1[0]-pos2[0])**2+(pos1[1]-pos2[1])**2)**0.5


def player_direction(obs):
    controlled_player_pos = obs['left_team'][obs['active']]
    controlled_player_dir = obs['left_team_direction'][obs['active']]
    x = controlled_player_pos[0]
    y = controlled_player_pos[1]
    dx = controlled_player_dir[0]
    dy = controlled_player_dir[1]
#      if x <= dx:
#         return 0
#     if x > dx:
#         return 1
    X_dir, Y_dir = dirsign(dx), dirsign(dy) # goes straight
    return directions[Y_dir][X_dir]


def do_toward(obs, action, CUR_DIR, x, y, tx, ty, just_x=False):
    try:
        xdir = dirsign(tx - x)
        ydir = dirsign(ty - y)
        SEL_DIR = directions[ydir][xdir]
        if just_x:
            # for quick action
            # todo: select closer dir by finding which ydir is currently
            Accepted_dirs = [directions[tmp][xdir] for tmp in [0,1,2]]
            if CUR_DIR not in Accepted_dirs:
                return SEL_DIR
        else:
            if SEL_DIR != CUR_DIR:
                return SEL_DIR
    except:
        pass
    
    if SEL_DIR not in obs['sticky_actions']:
        return SEL_DIR
    
    return action


def run_pass(left_team,right_team,x,y):
    # Go and Pass
    teammateL=0
    teammateR=0
    for i in range(len(left_team)):
        #is there a teamate close to left
        if left_team[i][0] >= x:
            if left_team[i][1] < y:
                if abs(left_team[i][1] - x) <.05:
                    teammateL=teammateL+1
        
        #is there a teamate to right
        if left_team[i][0] >= x:
            if left_team[i][1] > y:
                if abs(left_team[i][1] - x) <.05:
                    teammateR=teammateR+1
    #pass only close to goal
    if x >.75:
        if teammateL > 0 or teammateR > 0:
            return Action.ShortPass
    return Action.Right
    

@human_readable_agent
def agent(obs):
    
    CUR_DIR = player_direction(obs)
    left_team,right_team = obs['left_team'],obs['right_team']
    # our selected player
    controlled_player_pos = left_team[obs['active']]
    # player position
    x,y = controlled_player_pos[0],controlled_player_pos[1]
    #vector where ball is going
    ball_targetx=obs['ball'][0]+obs['ball_direction'][0]
    ball_targety=obs['ball'][1]+obs['ball_direction'][1]
    
    
    # special plays. new start
    if obs["game_mode"] == GameMode.Penalty:
        return Action.Shot
    if obs["game_mode"] == GameMode.Corner:
        if controlled_player_pos[0] > 0:
            return Action.Shot
    if obs["game_mode"] == GameMode.FreeKick:
        return Action.Shot
    if obs["game_mode"] == GameMode.GoalKick:
        return Action.HighPass
    
    if obs["game_mode"] in [GameMode.KickOff]:
        # start of game / after goal
        pass
    
    # SPRINT
    if  Action.Sprint not in obs['sticky_actions']:
        return Action.Sprint


    # Does the player we control have the ball?
    if obs['ball_owned_player'] == obs['active'] and obs['ball_owned_team'] == 0:
        
        if obs['right_team'][0][0] < 0.8:
            return Action.Shot
        
        if inside(controlled_player_pos, [[0.6, 1], [-0.2, 0.2]]):
            if x < obs['ball'][0]:
                return Action.Shot
        
        if inside(controlled_player_pos, [[0.2, 1], [-0.25, 0.25]]):
            if y>0:
                return do_toward(obs, Action.Shot, CUR_DIR, x, y, 1.02, 0.03)
            else:
                return do_toward(obs, Action.Shot, CUR_DIR, x, y, 1.02, -0.03)
        
        # if close to goal and too wide for shot pass the ball
        if x >.75:
            if abs(y) >.3:
                return Action.HighPass
            elif abs(y) >.20:
                return Action.LongPass
        
        if -0.2 < x < 0.2:
            return do_toward(obs, Action.ShortPass, CUR_DIR, x, y, 1.02, 0.03, True)
        
        # defense
        if x < -0.2:
            return Action.HighPass
            
        # from sides
        if abs(y) > 0.2:
            if x > 0:
                return Action.LongPass
            else:
                return Action.HighPass
        
        # which way should we run or pass
        return run_pass(left_team,right_team,x,y)
    else:
        #euclidian distance to the ball so we head off movement until very close
        e_dist=get_distance(obs['left_team'][obs['active']],obs['ball'])
        
        #if not close to ball move to where it is going
        if e_dist >.005:
            # Run where ball will be
            xdir = dirsign(ball_targetx - x)
            ydir = dirsign(ball_targety - y)
            return directions[ydir][xdir]
        else:
            #if close to ball go to ball
            return Action.Slide


# Don't Edit

In [ ]:
# A sample game: Our agent vs. do-nothing BOT
from kaggle_environments import make
env = make("football", configuration={"save_video": True, "scenario_name": "11_vs_11_kaggle", "running_in_notebook": True})
output = env.run(["/kaggle/working/submission.py","do_nothing"])[-1]
print('vs do nothing')
print('(THIS)Left player: reward = %s, status = %s, info = %s' % (output[0]['reward'], output[0]['status'], output[0]['info']))
print('(BOT)Right player: reward = %s, status = %s, info = %s' % (output[1]['reward'], output[1]['status'], output[1]['info']))
env.render(mode="human", width=600, height=400)

In [ ]:
!find . ! -name 'submission.py' -type d -exec rm -r -f {} +
!ls